### Import library

In [28]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Subset
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR100
transform = transforms.ToTensor()

# create training dataset
trainset = CIFAR100(root='./data', train=True, download=True, transform=transform)

Files already downloaded and verified


In [80]:
NUM_CLIENTS = 50

def load_datasets(num_clients: int):
    # Download and transform CIFAR-100 (train and test)
    transform = transforms.Compose(
      [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR100("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR100("./dataset", train=False, download=True, transform=transform)

    #Create the subsets
    trainloaders = []
    valloaders = []
    for x in range(0,num_clients):
        # We create a tensor that has `True` at an index if the sample belongs to class
        idx0 = torch.tensor(trainset.targets) == x
        idx1 = torch.tensor(trainset.targets) == x+50
        idx = idx0 | idx1
        idx0t = torch.tensor(testset.targets) == x
        idx1t = torch.tensor(testset.targets) == x+50
        idxt = idx0t | idx1t
        #We then need to convert this into a list of indices at which we have True.
        train_indices = idx.nonzero().reshape(-1)
        val_indices = idxt.nonzero().reshape(-1)
        #append new subset
        ds_train=torch.utils.data.Subset(trainset,  train_indices)
        ds_val= torch.utils.data.Subset(testset,  val_indices)
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32, shuffle=True))
    return trainloaders, valloaders


In [81]:
trainloaders, valloaders = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


In [82]:
type(trainloaders[0])

torch.utils.data.dataloader.DataLoader